In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.xls')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1473,tt0429589,50000000,55181129,The Ant Bully,Julia Roberts|Meryl Streep|Nicolas Cage|Paul G...,John A. Davis,The battle for the lawn is on.,Fed up with being targeted by the neighborhood...,88,Fantasy|Adventure|Animation|Comedy|Family,Warner Bros. Pictures|DNA Films|Legendary Pict...,7/27/2006,5.5,2006
991,tt1392170,75000000,691210692,The Hunger Games,Jennifer Lawrence|Josh Hutcherson|Liam Hemswor...,Gary Ross,May The Odds Be Ever In Your Favor.,Every year in the ruins of what was once North...,142,Science Fiction|Adventure|Fantasy,Lionsgate|Color Force,3/12/2012,6.7,2012
341,tt1114677,35000000,155545279,Hannah Montana: The Movie,Miley Cyrus|Billy Ray Cyrus|Emily Osment|Jason...,Peter Chelsom,She has the best of both worlds...now she has ...,When Miley Stewart (aka pop-star Hannah Montan...,102,Comedy|Drama|Family|Music|Romance,Walt Disney Pictures|It's a Laugh Productions|...,4/10/2009,5.8,2009
1110,tt0311113,150000000,212011111,Master and Commander: The Far Side of the World,Russell Crowe|Paul Bettany|James D'Arcy|Billy ...,Peter Weir,The Courage to Do the Impossible Lies in the H...,After an abrupt and violent encounter with a F...,138,Adventure,Universal Pictures|Twentieth Century Fox Film ...,11/14/2003,6.6,2003
1660,tt0324127,27000000,13000000,Suspect Zero,Carrie-Anne Moss|Aaron Eckhart|Ben Kingsley|Ha...,E. Elias Merhige,Who's next?,"A killer is on the loose, and an FBI agent sif...",99,Crime|Thriller,Paramount Pictures|Cruise/Wagner Productions|L...,8/27/2004,5.1,2004


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

data['profit'] = data['revenue'] - data['budget'] # добавим в dataset колонку profit для 6-го вопроса


def column_clean(s):
    """функция для "очистки" колонки от символов |"""
    result = s.split('|')
    return result


def list_to_str(s):
    """функция для перевода значений колонки в str"""
    result = ','.join(s)
    return result


def unique_list_creator(s):
    """функция формирования списка уникальных позиций"""
    for name in item:
        if name not in s:
            s.append(name)
    return s


actors_list=[] #создадим список уникальных актеров
data.cast=data.cast.apply(column_clean)
for item in data.cast:
    for name in item:
        unique_list_creator(actors_list) 

data['cast_list'] = data.cast #создадим столбец со списком актеров в виде списка
data.cast=data.cast.apply(list_to_str)

genres_list=[] #создадим список уникальных жанров
data.genres=data.genres.apply(column_clean)
for item in data.genres:
    for name in item:
        unique_list_creator(genres_list)
data['genres_list'] = data.genres #создадим столбец со списком жанров в виде списка
data.genres=data.genres.apply(list_to_str)

directors_list=[] # создадим список уникальных режисеров
data.director=data.director.apply(column_clean)
for item in data.director:
    for name in item:
        unique_list_creator(directors_list)
data.director=data.director.apply(list_to_str)

production_companies_list=[] # создадим список уникальных студий
data.production_companies=data.production_companies.apply(column_clean)
for item in data.production_companies:
    for name in item:
        unique_list_creator(production_companies_list)
data.production_companies=data.production_companies.apply(list_to_str)

months_list = ['январь','февраль','март','апрель','май','июнь','июль','август','сентябрь','октябрь','ноябрь','декабрь'] 
# создадим список месяцев

def release_month(date):
    """функция для выделения месяца выпуска фильма"""
    if date!='nan':
        result=date[0:date.find('/')]
        for i in range(1,13):
            if int(result) == i:
                result=months_list[i-1]
                return result


data['release_month'] =  data.release_date.apply(release_month)
data['original_title_len'] = data.original_title.apply(lambda x: len(x)) #создаем столбец с длиной названий фильмов


def words_count(s):
    """функция для подсчета количества слов""" 
    result = len(str(s).split())
    return result


data['overview_words'] = data.overview.apply(words_count) #создаем столбец с количеством слов в описании

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [5]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '723. Pirates of the Caribbean: On Stranger Tides (tt1298650) (+)'
# если ответили верно, можете добавить комментарий со значком "+"

In [6]:
data[data.budget==data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,cast_list,genres_list,release_month,original_title_len,overview_words
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,"Johnny Depp,PenÃ©lope Cruz,Geoffrey Rush,Ian M...",Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,"Adventure,Action,Fantasy","Walt Disney Pictures,Jerry Bruckheimer Films,M...",5/11/2011,6.3,2011,641683000,"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...","[Adventure, Action, Fantasy]",май,43,75


ВАРИАНТ 2

In [7]:
d = data.sort_values(['budget'],ascending=False).head(5)
display(d)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,cast_list,genres_list,release_month,original_title_len,overview_words
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,"Johnny Depp,PenÃ©lope Cruz,Geoffrey Rush,Ian M...",Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,"Adventure,Action,Fantasy","Walt Disney Pictures,Jerry Bruckheimer Films,M...",5/11/2011,6.3,2011,641683000,"[Johnny Depp, PenÃ©lope Cruz, Geoffrey Rush, I...","[Adventure, Action, Fantasy]",май,43,75
1669,tt0449088,300000000,961000000,Pirates of the Caribbean: At World's End,"Johnny Depp,Orlando Bloom,Keira Knightley,Geof...",Gore Verbinski,"At the end of the world, the adventure begins.","Captain Barbossa, long believed to be dead, ha...",169,"Adventure,Fantasy,Action","Walt Disney Pictures,Jerry Bruckheimer Films,S...",5/19/2007,6.8,2007,661000000,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Adventure, Fantasy, Action]",май,40,39
14,tt2395427,280000000,1405035767,Avengers: Age of Ultron,"Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo...",Joss Whedon,A New Age Has Come.,When Tony Stark tries to jumpstart a dormant p...,141,"Action,Adventure,Science Fiction","Marvel Studios,Prime Focus,Revolution Sun Studios",4/22/2015,7.4,2015,1125035767,"[Robert Downey Jr., Chris Hemsworth, Mark Ruff...","[Action, Adventure, Science Fiction]",апрель,23,69
1014,tt0401729,260000000,284139100,John Carter,"Taylor Kitsch,Lynn Collins,Mark Strong,Willem ...",Andrew Stanton,Lost in Our World. Found in Another.,Civil War vet John Carter is transplanted to M...,132,"Action,Adventure,Fantasy,Science Fiction",Walt Disney Pictures,3/7/2012,6.0,2012,24139100,"[Taylor Kitsch, Lynn Collins, Mark Strong, Wil...","[Action, Adventure, Fantasy, Science Fiction]",март,11,50
380,tt0398286,260000000,591794936,Tangled,"Zachary Levi,Mandy Moore,Donna Murphy,Ron Perl...","Nathan Greno,Byron Howard",They're taking adventure to new lengths.,When the kingdom's most wanted-and most charmi...,100,"Animation,Family","Walt Disney Pictures,Walt Disney Animation Stu...",11/24/2010,7.3,2010,331794936,"[Zachary Levi, Mandy Moore, Donna Murphy, Ron ...","[Animation, Family]",ноябрь,7,84


# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '1157. Gods and Generals (tt0279111) (+)'

In [9]:
data[data.runtime == data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,cast_list,genres_list,release_month,original_title_len,overview_words
1157,tt0279111,56000000,12923936,Gods and Generals,"Stephen Lang,Jeff Daniels,Robert Duvall,Kevin ...",Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,"Drama,History,War","Turner Pictures,Antietam Filmworks",2/21/2003,5.8,2003,-43076064,"[Stephen Lang, Jeff Daniels, Robert Duvall, Ke...","[Drama, History, War]",февраль,17,48


# 3. Какой из фильмов самый короткий (в минутах)?





In [10]:
answers['3'] = '768. Winnie the Pooh (tt1449283) (+)'
data[data.runtime==data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,cast_list,genres_list,release_month,original_title_len,overview_words
768,tt1449283,30000000,14460000,Winnie the Pooh,"Jim Cummings,Travis Oates,Jim Cummings,Bud Luc...","Stephen Anderson,Don Hall",Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,"Animation,Family","Walt Disney Pictures,Walt Disney Animation Stu...",4/13/2011,6.8,2011,-15540000,"[Jim Cummings, Travis Oates, Jim Cummings, Bud...","[Animation, Family]",апрель,15,52


# 4. Какова средняя длительность фильмов?


In [11]:
answers['4'] = '110 (+)'
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [12]:
answers['5'] = '107 (+)'
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [13]:
answers['6'] = '239. Avatar (tt0499549) (+)'
data[data.profit==data.profit.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,cast_list,genres_list,release_month,original_title_len,overview_words
239,tt0499549,237000000,2781505847,Avatar,"Sam Worthington,Zoe Saldana,Sigourney Weaver,S...",James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,"Action,Adventure,Fantasy,Science Fiction","Ingenious Film Partners,Twentieth Century Fox ...",12/10/2009,7.1,2009,2544505847,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Action, Adventure, Fantasy, Science Fiction]",декабрь,6,28


# 7. Какой фильм самый убыточный? 

In [14]:
answers['7'] = '1245. The Lone Ranger (tt1210819) (+)'
data[data.profit==data.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,cast_list,genres_list,release_month,original_title_len,overview_words
1245,tt1210819,255000000,89289910,The Lone Ranger,"Johnny Depp,Armie Hammer,William Fichtner,Hele...",Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,"Action,Adventure,Western","Walt Disney Pictures,Jerry Bruckheimer Films,I...",7/3/2013,6.0,2013,-165710090,"[Johnny Depp, Armie Hammer, William Fichtner, ...","[Action, Adventure, Western]",июль,15,68


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [15]:
answers['8'] = '1478 (+)'
data[(data.profit>0)].imdb_id.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [16]:
answers['9'] = '599. The Dark Knight (tt0468569) (+)'
r = data[(data.release_year==2008)].revenue.max() #определяем максимальную сумму кассовых сборов в 2018 году
data[data.revenue==r]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,cast_list,genres_list,release_month,original_title_len,overview_words
599,tt0468569,185000000,1001921825,The Dark Knight,"Christian Bale,Michael Caine,Heath Ledger,Aaro...",Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,"Drama,Action,Crime,Thriller","DC Comics,Legendary Pictures,Warner Bros.,Syncopy",7/16/2008,8.1,2008,816921825,"[Christian Bale, Michael Caine, Heath Ledger, ...","[Drama, Action, Crime, Thriller]",июль,15,67


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [17]:
answers['10'] = '1245. The Lone Ranger (tt1210819) (+)'
r = data[(data.release_year>=2012)&(data.release_year<=2014)].profit.min() #определяем максимальную сумму убытка по фильмам\
#c 2012 по 2014 год
data['profit'] = data['revenue']-data['budget']
data[data.profit==r]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,cast_list,genres_list,release_month,original_title_len,overview_words
1245,tt1210819,255000000,89289910,The Lone Ranger,"Johnny Depp,Armie Hammer,William Fichtner,Hele...",Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,"Action,Adventure,Western","Walt Disney Pictures,Jerry Bruckheimer Films,I...",7/3/2013,6.0,2013,-165710090,"[Johnny Depp, Armie Hammer, William Fichtner, ...","[Action, Adventure, Western]",июль,15,68


# 11. Какого жанра фильмов больше всего?

In [18]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
answers['11'] = 'Drama (+)'
genres_count = {genre: data[data['genres'].str.contains(genre)].genres.count() for genre in genres_list}\
#создаем словарь для записи жанров и количества фильмов
c=Counter(genres_count)
print(c.most_common(1))

[('Drama', 782)]


ВАРИАНТ 2

In [19]:
genres=list(data.genres_list) #создадим список жанров
cnt=Counter()
for item in genres:
    for word in item:
        cnt[word] += 1
print(cnt.most_common(1))

[('Drama', 782)]


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [20]:
answers['12'] = 'Drama (+)'
genres_count = {genre: data[(data.genres.str.contains(genre))&(data['profit']>0)].profit.count() for genre in genres_list}\
#создаем словарь для записи жанров и количества фильмов
c=Counter(genres_count)
display(c.most_common(1))

[('Drama', 560)]

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [21]:
answers['13'] = 'Peter Jackson (+)'
director_revenue_count = {director: data[data.director.str.contains(director)].revenue.sum() for director in directors_list}\
#создаем словарь для записи режиссеров и кассовых сборов
c=Counter(director_revenue_count)
display(c.most_common(1))

[('Peter Jackson', 6490593685)]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [22]:
answers['14'] = 'Robert Rodriguez (+)'
director_action_count = {director: data[(data.genres.str.contains('Action'))&(data.director.str.contains(director))].imdb_id.count()\
for director in directors_list} #создаем словарь для записи режиссеров и количества фильмов
c=Counter(director_action_count)
display(c.most_common(1))

[('Robert Rodriguez', 9)]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [23]:
answers['15'] = 'Chris Hemsworth (+)'
actors_revenues = {name: data[(data.cast.str.contains(name)) & (data.release_year == 2012)].revenue.sum()\
for name in actors_list} #создаем словарь для записи актеров и количества фильмов
c=Counter(actors_revenues)
print(c.most_common(1))

[('Chris Hemsworth', 2027450773)]


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [24]:
answers['16'] = 'Matt Damon (+)'
actors_highbudget_movies = {name: data[(data['cast'].str.contains(name)) & (data.budget>data.budget.mean())].imdb_id.count()\
for name in actors_list} #создаем словарь для записи актеров и количества фильмов
c=Counter(actors_highbudget_movies)
print(c.most_common(1))

[('Matt Damon', 18)]


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [25]:
answers['17'] = 'Action (+)'
Nicolas_Cage_count={genre: data[(data['cast'].str.contains('Nicolas Cage')) & (data['genres'].str.contains(genre))].imdb_id.count()\
for genre in genres_list} #создаем словарь для записи жанров и количества фильмов
c=Counter(Nicolas_Cage_count)
print(c.most_common(1))

[('Action', 17)]


# 18. Самый убыточный фильм от Paramount Pictures

In [26]:
answers['18'] = 'K-19: The Widowmaker (tt0267626) (+)'
d = data[data['production_companies'].str.contains('Paramount Pictures')].profit.min()
data.loc[(data['production_companies'].str.contains('Paramount Pictures')) & (data.profit==d)].original_title

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [27]:
answers['19'] = '2015 (+)'
df = data.groupby(['release_year'])['revenue'].agg(['sum']).sort_values(by='sum',ascending=False)
df.head(1)

,sum
release_year,
2015,25449202382


# 20. Какой самый прибыльный год для студии Warner Bros?

In [28]:
answers['20'] = '2014 (+)'
df = data[data['production_companies'].str.contains('Warner Bros')]
df = df.groupby(['release_year'])['profit'].agg(['sum']).sort_values(by='sum',ascending=False)
df.head(1)

,sum
release_year,
2014,2295464519


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [29]:
answers['21'] = 'сентябрь (+)'
df = data.groupby(['release_month'])['imdb_id'].agg(['count']).sort_values(by='count',ascending=False)
df.head(1)

,count
release_month,
сентябрь,227


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [30]:
answers['22'] = '450 (+)'
df = data.groupby(['release_month'])['imdb_id'].agg(['count']).sort_values(by='count',ascending=False)
count = df.loc['июнь'] + df.loc['июль'] + df.loc['август']
int(count)

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [31]:
answers['23'] = 'Peter Jackson(+)'
director_winter_count={director:  data[(data['director'].str.contains(director)) & \
(data['release_month'].str.contains('декабрь|январь|февраль'))].imdb_id.count() for director in directors_list}\
#создаем словарь для записи режиссеров и количества фильмов
c=Counter(director_winter_count)
print(c.most_common(1))

[('Peter Jackson', 7)]


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [32]:
answers['24'] = 'Four By Two Productions (+)'
production_companies_simbols={name: data[data.production_companies.str.contains(name, regex=False)].original_title_len.mean()\
for name in production_companies_list} #создаем словарь для записи студий и количества символов
c=Counter(production_companies_simbols)
print(c.most_common(1))

[('Four By Two Productions', 83.0)]


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [33]:
answers['25'] = 'Midnight Picture Show (+)'
production_companies_words={name: data[data['production_companies'].str.contains(name, regex=False)].overview_words.mean()\
for name in production_companies_list} #создаем словарь для записи студий и среднего количества слов
c=Counter(production_companies_words)
print(c.most_common(1))

[('Midnight Picture Show', 175.0)]


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [34]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave(+)'
dn = np.percentile(data['vote_average'],99,axis=0)
print(data[data['vote_average']>=dn].original_title)

9                                              Inside Out
28                                              Spotlight
34                                                   Room
118                                          Interstellar
119                               Guardians of the Galaxy
124                                            Big Hero 6
125                                    The Imitation Game
128                                             Gone Girl
138                              The Grand Budapest Hotel
155                              The Theory of Everything
177                                The Fault in Our Stars
283                                            Mr. Nobody
316                                              3 Idiots
370                                             Inception
496     The Lord of the Rings: The Fellowship of the Ring
599                                       The Dark Knight
863                 The Lord of the Rings: The Two Towers
872           

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [35]:
answers['27'] = 'Daniel Radcliffe, Rupert Grint(+)'
cast=list(data.cast_list) #создадим список актеров 
cnt = Counter()
for item in cast:
    for pair in combinations(item,2):
        cnt[pair] += 1
print(cnt.most_common(5))

[(('Daniel Radcliffe', 'Rupert Grint'), 8), (('Daniel Radcliffe', 'Emma Watson'), 8), (('Rupert Grint', 'Emma Watson'), 7), (('Ben Stiller', 'Owen Wilson'), 6), (('Johnny Depp', 'Helena Bonham Carter'), 6)]


# Submission

In [36]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '723. Pirates of the Caribbean: On Stranger Tides (tt1298650) (+)',
 '2': '1157. Gods and Generals (tt0279111) (+)',
 '3': '768. Winnie the Pooh (tt1449283) (+)',
 '4': '110 (+)',
 '5': '107 (+)',
 '6': '239. Avatar (tt0499549) (+)',
 '7': '1245. The Lone Ranger (tt1210819) (+)',
 '8': '1478 (+)',
 '9': '599. The Dark Knight (tt0468569) (+)',
 '10': '1245. The Lone Ranger (tt1210819) (+)',
 '11': 'Drama (+)',
 '12': 'Drama (+)',
 '13': 'Peter Jackson (+)',
 '14': 'Robert Rodriguez (+)',
 '15': 'Chris Hemsworth (+)',
 '16': 'Matt Damon (+)',
 '17': 'Action (+)',
 '18': 'K-19: The Widowmaker (tt0267626) (+)',
 '19': '2015 (+)',
 '20': '2014 (+)',
 '21': 'сентябрь (+)',
 '22': '450 (+)',
 '23': 'Peter Jackson(+)',
 '24': 'Four By Two Productions (+)',
 '25': 'Midnight Picture Show (+)',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave(+)',
 '27': 'Daniel Radcliffe, Rupert Grint(+)'}

In [37]:
# и убедиться что ни чего не пропустил)
len(answers)

27